# Assistente de Concessionária — Indicadores & Métricas

## 1 - Preparar o conjunto de dados

### 1.1 - Carregar conjunto de dados

In [1]:
import pandas as pd

In [2]:
df_mensagens = pd.read_csv(
    'mensagens.csv',
    sep=';',
    index_col=False
)

In [3]:
df_mensagens.head()

,mensagem,data,horario,intencoes,entidades
0,dinheiro,2021-05-20,21:23,NaN,NaN
1,carro,2021-05-20,21:23,#recomendar-modelo,NaN
2,fusion,2021-05-20,21:24,NaN,@modelo-carro:Ford Fusion Hybrid 2021
3,ford,2021-05-20,21:24,NaN,NaN
4,aff,2021-05-20,21:24,NaN,NaN


### 1.2 - Checar integridade dos campos de texto

Checar se algum dos campos de texto possui um ou mais elementos vazios.

In [4]:
def serie_possui_str_vazia(serie: pd.Series):
    return serie.apply(lambda x: not x).any()

In [5]:
for col in ['mensagem', 'data', 'horario']:
    print(f'{col}:', serie_possui_str_vazia(df_mensagens['mensagem']))

mensagem: False
data: False
horario: False


### 1.3 - Converter tipos de dados

Converter datas e horários para o tipo `datetime`.

In [6]:
from datetime import datetime

In [7]:
df_mensagens['data_horario'] = df_mensagens[['data', 'horario']].apply(
    lambda row: datetime.fromisoformat(' '.join(row.values)),
    axis=1
)
df_mensagens.drop(['data', 'horario'], axis=1, inplace=True)

df_mensagens['data_horario'].head()

0   2021-05-20 21:23:00
1   2021-05-20 21:23:00
2   2021-05-20 21:24:00
3   2021-05-20 21:24:00
4   2021-05-20 21:24:00
Name: data_horario, dtype: datetime64[ns]

Converter intenções e entidades em listas.

In [8]:
df_mensagens.fillna('', inplace=True)

df_mensagens.head()

,mensagem,intencoes,entidades,data_horario
0,dinheiro,,,2021-05-20 21:23:00
1,carro,#recomendar-modelo,,2021-05-20 21:23:00
2,fusion,,@modelo-carro:Ford Fusion Hybrid 2021,2021-05-20 21:24:00
3,ford,,,2021-05-20 21:24:00
4,aff,,,2021-05-20 21:24:00


In [9]:
df_mensagens['intencoes'] = df_mensagens['intencoes'].str.split(',')

df_mensagens['intencoes'].head()

0                      []
1    [#recomendar-modelo]
2                      []
3                      []
4                      []
Name: intencoes, dtype: object

In [10]:
df_mensagens['entidades'] = df_mensagens['entidades'].str.split(',')

df_mensagens['entidades'].head()

0                                         []
1                                         []
2    [@modelo-carro:Ford Fusion Hybrid 2021]
3                                         []
4                                         []
Name: entidades, dtype: object

In [11]:
df_mensagens['intencoes'].apply(len).max()

1

In [12]:
df_mensagens['entidades'].apply(len).max()

2

Quebrar as entidades em duplas entidade–valor.

In [14]:
df_mensagens['entidades'] = df_mensagens['entidades'].apply(
    lambda l: [tuple(termos.split(':')) for termos in l]
)

df_mensagens['entidades'].head()

0                                         [(,)]
1                                         [(,)]
2    [(@modelo-carro, Ford Fusion Hybrid 2021)]
3                                         [(,)]
4                                         [(,)]
Name: entidades, dtype: object

In [15]:
df_mensagens['entidades'].apply(
    lambda l: all((len(e) in [0,2]) for e in l)
).all()

False

In [16]:
df_mensagens[
    df_mensagens['entidades'].apply(
        lambda l: any(len(e) not in [0,2] for e in l)
    )
]

,mensagem,intencoes,entidades,data_horario
0,dinheiro,[],"[(,)]",2021-05-20 21:23:00
1,carro,[#recomendar-modelo],"[(,)]",2021-05-20 21:23:00
3,ford,[],"[(,)]",2021-05-20 21:24:00
4,aff,[],"[(,)]",2021-05-20 21:24:00
8,Que categorias você tem?,[],"[(,)]",2021-05-22 17:21:00
9,Liste por categoria,[#avaliar-por-categoria],"[(,)]",2021-05-22 17:21:00
11,Liste por categoria.,[#avaliar-por-categoria],"[(,)]",2021-05-22 17:22:00
13,Que tal o mach-e?,[#avaliar-por-finalidade],"[(,)]",2021-05-22 17:22:00
15,Sim,[#confirmacao],"[(,)]",2021-05-22 17:23:00
16,Sim,[#confirmacao],"[(,)]",2021-05-22 17:23:00
